Check dataset row count

In [17]:
import pandas as pd

df = pd.read_json("../data//full.json")
print(df.shape[0])
df = pd.read_json("../data//2021-09-14//pureness2.json")
print(df.shape[0])
print(df.head(-10))

14744
10000
           Id  BreedCount  Pureness  Class       Eyes           Ears  \
0      715640           2         2    Bug      Clear          Lotus   
1     3788301           2         2  Plant      Topaz        Inkling   
2     1497560           2         2  Beast      Mavis         Pogona   
3     5726772           1         2  Beast      Topaz           Nimo   
4     1860143           3         2  Beast      Gecko    Peace Maker   
...       ...         ...       ...    ...        ...            ...   
9985  2684536           2         2    Bug     Chubby         Pogona   
9986  2684955           2         2    Bug  Sleepless        Tassels   
9987  2684989           2         2    Bug    Blossom        Tassels   
9988  2681532           3         2    Bug      Clear        Inkling   
9989  2679071           2         2    Bug     Chubby  Ear Breathing   

              Back            Mouth        Horn         Tail   Price  
0     Watering Can          Catfish     Lagging  Nut

Feature engineering

In [ ]:
import pandas as pd

leaderboard = pd.read_csv('..\leaderboard\leaderboard.csv')
# print(leaderboard.head())
df=pd.read_json('..\data\\2021-09-13\\full.json')
# print(df.head())


In [ ]:
def assign_score(row, frecuencies, combo):
  try:
    return frecuencies[row["Class"],row[combo]]
  except KeyError:
      return 0
  
def normalize(series):
    return (series - series.min()) / (series.max() - series.min())

def card_score(df, leaderboard):
  for part in ['Back','Mouth','Horn','Tail']:  
      frecuencies = normalize( leaderboard.loc[:,['Class',part]].value_counts() )
      df.loc[:,f'{part}_score'] = df.apply( lambda df: assign_score(df,frecuencies,part), axis=1)
card_score(df,leaderboard)
df.head(10)

In [ ]:
def combo_score(df, leaderboard):
  for dtset in [df, leaderboard]:
    dtset["Back-Mouth"] =dtset["Back"]+"/"+dtset["Mouth"]
    dtset["Back-Horn"]  =dtset["Back"]+"/"+dtset["Horn"]
    dtset["Back-Tail"]  =dtset["Back"]+"/"+dtset["Tail"]
    dtset["Mouth-Horn"] =dtset["Mouth"]+"/"+dtset["Horn"]
    dtset["Mouth-Tail"] =dtset["Mouth"]+"/"+dtset["Tail"]
    dtset["Horn-Tail"]  =dtset["Horn"]+"/"+dtset["Tail"]
  for combo in ["Back-Mouth","Back-Horn","Back-Tail","Mouth-Horn","Mouth-Tail","Horn-Tail"]:  
      frecuencies = normalize( leaderboard.loc[:,['Class',combo]].value_counts() )
      df.loc[:,f'{combo}_score'] = df.apply( lambda df: assign_score(df,frecuencies,combo), axis=1)
      for dtset in [df, leaderboard]:
        dtset.drop(combo,axis=1,inplace=True)
combo_score(df, leaderboard)
df.head()

In [ ]:
def tricombo_score(df, leaderboard):
  for dtset in [df, leaderboard]:
    dtset["Back-Mouth-Horn"] = dtset["Back"] +"/"+dtset["Mouth"]+"/"+dtset["Horn"]
    dtset["Back-Mouth-Tail"] = dtset["Back"] +"/"+dtset["Mouth"]+"/"+dtset["Tail"]
    dtset["Back-Horn-Tail"]  = dtset["Back"] +"/"+dtset["Horn"] +"/"+dtset["Tail"]
    dtset["Mouth-Horn-Tail"] = dtset["Mouth"]+"/"+dtset["Horn"] +"/"+dtset["Tail"]
  for tricombo in ["Back-Mouth-Horn","Back-Mouth-Tail","Back-Horn-Tail","Mouth-Horn-Tail"]:  
        frecuencies = normalize( leaderboard.loc[:,['Class',tricombo]].value_counts() )
        df.loc[:,f'{tricombo}_score'] = df.apply( lambda df: assign_score(df,frecuencies,tricombo), axis=1)
        for dtset in [df, leaderboard]:
          dtset.drop(tricombo,axis=1,inplace=True)
tricombo_score(df, leaderboard)
df.head()

In [ ]:
  def build_score(df, leaderboard):
    for dtset in [df, leaderboard]:
      dtset["Build"] = dtset["Back"]+"/"+dtset["Mouth"]+"/"+dtset["Horn"]+"/"+dtset["Tail"] 
    frecuencies = normalize( leaderboard.loc[:,["Class","Build"]].value_counts() )
    df.loc[:,'Build_score'] = df.apply( lambda df: assign_score(df,frecuencies,'Build'), axis=1)
    for dtset in [df, leaderboard]:
      dtset.drop('Build',axis=1,inplace=True)
build_score(df, leaderboard)
df.head()
